In [ ]:
import requests
import json
import pandas as pd
import sys

In [ ]:
# Задаем переменные

# url, по которому будем забирать из API MyTarget статистику по кампаниям
# day означает, что получаем статистику по дням
url = "https://target.my.com/api/v2/statistics/banners/day.json"

# конечно, токен лучше не передавать в исходном виде, но здесь это допустимо.
access_token = 'xxxxx'

# формируем заголовок запроса
headers = {"Authorization": "Bearer " + access_token}

# диапазон дат периода, по которому получаем статистику
date_from = '2021-01-10'
date_to = '2021-01-10'

# формируем список параметров
# all  означает, что выгружаем всю статистику
params = {
    "date_from": date_from,
    "date_to": date_to,
    "metrics": 'all',
}

In [ ]:
# отправляем запрос, если происходит ошибка, выводим ее
response = requests.get(url, params=params, headers=headers)
if response.status_code != 200:
    print("Failed to load statistics:", response.status_code, response.json())
    sys.exit(1)
    
load_data = response.json()

In [ ]:
# преобразем получившийся json в датафрейм

processed_rows = []

all_stat = load_data["items"]
for one_ad_stat in all_stat:
    ad_id = one_ad_stat["id"]
    one_ad_all_dates_stat = one_ad_stat["rows"]

    for one_ad_one_date_stat in one_ad_all_dates_stat:

        one_ad_one_date_stat.update({
            "id": ad_id,
        })

        row = pd.json_normalize(one_ad_one_date_stat, sep="_").to_dict("records")
        processed_rows += row

report_df = pd.DataFrame(processed_rows)

In [ ]:
report_df.head()